In [ ]:
import torch, cv2, copy
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader
from typing import Iterable
from torchvision.models.resnet import BasicBlock, Bottleneck
from data_preparation.dataset import FrameDataset
from model.architecture import GWDetectionNET
from metrics.criterion import GWDetectionCriterion
from pipelines.train import TrainingPipeline
from matplotlib import pyplot as plt

In [ ]:
def prepare_img_and_bbox(
        img: torch.Tensor, 
        bbox: torch.Tensor, 
        scaled_bbox: bool=True,
        xy_centered: bool=True):
    img = (img * 255).type(torch.uint8).squeeze().numpy()

    if bbox.shape[0] == 4:
        x, y, w, h = bbox
    else:
        _, x, y, w, h = bbox

    if scaled_bbox:
        x *= img.shape[1]
        y *= img.shape[0]
        w *= img.shape[1]
        h *= img.shape[0]

    if xy_centered:
        x = x-(w/2)
        y = y-(h/2)
    bbox = np.array([x, y, w, h]).astype(int)
    return img, bbox

def draw_bbox(img :np.ndarray, bbox: Iterable[int], single_channel: bool=True):
    if bbox.shape[0] == 4:
        x, y, w, h = bbox
    else:
        _, x, y, w, h = bbox
    ch = (0, 200, 90) if not single_channel else (255)
    return cv2.rectangle(img, (x, y), (x+w, y+h), ch, 2)


In [ ]:
train_data_path = "data/Train_data"
validation_data_path = "data/Validation_data"
test_data_path = "data/Test_data"
img_size = (224, 224)

train_dataset = FrameDataset(train_data_path, grayscale=True, img_loc="png", size=img_size)
validation_dataset = FrameDataset(validation_data_path, grayscale=True, img_loc="Image", size=img_size)
test_dataset = FrameDataset(test_data_path, grayscale=True, img_loc="Image", size=img_size)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
train_sample_img, train_sample_bbox = train_dataset[0]
validation_sample_img, validation_sample_bbox = validation_dataset[0]
test_sample_img, test_sample_bbox = test_dataset[0]

print(f"Sample Images (shape: {train_sample_img.shape})")
axs[0].imshow(train_sample_img.squeeze(), cmap="gray")
axs[1].imshow(validation_sample_img.squeeze(), cmap="gray")
axs[2].imshow(test_sample_img.squeeze(), cmap="gray")
plt.show()

train_sample_img, train_sample_bbox = prepare_img_and_bbox(train_sample_img, train_sample_bbox)
validation_sample_img, validation_sample_bbox = prepare_img_and_bbox(validation_sample_img, validation_sample_bbox)
test_sample_img, test_sample_bbox = prepare_img_and_bbox(test_sample_img, test_sample_bbox)


bbox_train_sample_img = draw_bbox(copy.deepcopy(train_sample_img), train_sample_bbox)
bbox_validation_sample_img = draw_bbox(copy.deepcopy(validation_sample_img), validation_sample_bbox)
bbox_test_sample_img = draw_bbox(copy.deepcopy(test_sample_img), test_sample_bbox)

print("\nSample Images with Bounding Boxes")
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
axs[0].imshow(bbox_train_sample_img, cmap="gray")
axs[1].imshow(bbox_validation_sample_img, cmap="gray")
axs[2].imshow(bbox_test_sample_img, cmap="gray")
plt.show()


In [ ]:
batch_size = 16
num_workers = 4

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [ ]:
# model params
in_channels = 1
n_classes = 0
dropout = 0.3
block = BasicBlock
block_layers = [2, 2, 2, 2]
pretrained_backbone = None #resnet18
last_fmap_ch = 512
anchor_path = "anchors/anchors.json"

# loss function params
iou_threshold = 0.5

# optimizer params
lr = 1e-4
min_lr = 1e-6
weight_decay = 7e-6
betas = (0.9, 0.9999)
T_0 = 40
T_mult = 2

# pipeline params
device = "cuda" if torch.cuda.is_available() else "cpu"
model_folder = "saved_model"
model_name = f"gw_detection_model.pth.tar"

In [ ]:
torch.manual_seed(3407)
torch.cuda.manual_seed(3407)
np.random.seed(3407)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
model = GWDetectionNET(
    in_channels, 
    img_size=img_size,
    anchors_path=anchor_path, 
    n_classes=n_classes, 
    block=block, 
    block_layers=block_layers,
    pretrained_resnet_backbone=pretrained_backbone, 
    last_fmap_ch=last_fmap_ch)
lossfunc = GWDetectionCriterion(iou_threshold=iou_threshold)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

# training pipeline
train_pipeline = TrainingPipeline(
    model, 
    lossfunc, 
    optimizer, 
    device=device, 
    dirname=model_folder, 
    filename=model_name,
    save_metrics=True
)

# lr scheduler
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    train_pipeline.optimizer, T_0=T_0, T_mult=T_mult, eta_min=min_lr, verbose=True,
)

In [9]:
epochs = 120

import warnings
warnings.filterwarnings("ignore")

for epoch in range(epochs):
    print(f"epoch: {epoch+1} / {epochs}")
    train_pipeline.train(train_dataloader, verbose=True)
    train_pipeline.evaluate(validation_dataloader, verbose=True)
    lr_scheduler.step()
    print("-"*130)

train_pipeline.plot_metrics("train")
train_pipeline.plot_metrics("eval")

9it [02:02, 13.58s/it]


KeyboardInterrupt: 